In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import keras
from keras.layers import Dense,Conv2D, BatchNormalization, Activation,Dropout
from keras.layers import AveragePooling2D, Input, Flatten,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
import keras.backend as K
import cv2
import seaborn as sns
import PIL
import skimage
from keras.models import Model
%matplotlib inline

Using TensorFlow backend.


In [3]:
def data_generate(data_path, batch_size, argumentation = True, shuffle=False):
    while True:
        fid = h5py.File(data_path, 'r')
        data_len = fid['sen1'].shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = np.array((fid['label'][i * batch_size: (i+1)*batch_size]))
                x_b1 = fid['sen1'][i * batch_size: (i+1)*batch_size]
                x_b2 = fid['sen2'][i * batch_size: (i+1)*batch_size]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([1,2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,::-1,:,:]
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                x_b1 = fid['sen1'][i*batch_size:]
                x_b2 = fid['sen2'][i*batch_size:]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                y_b = np.array((fid['label'][i * batch_size:]))
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3] , y_b)

In [4]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [6]:
def resnet_v2(input_tensor, depth, num_classes=17):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    #inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=input_tensor,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    #y = Flatten()(x)
    #outputs = Dense(num_classes,
     #               activation='softmax',
     #               kernel_initializer='he_normal')(y)

    # Instantiate model.
    #model = Model(inputs=inputs, outputs=outputs)
    return x

In [7]:
import keras.backend as K
import tensorflow as tf
def focal_loss(classes_num, gamma=2., alpha=.25, e=0.1):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''
        import tensorflow as tf
        from tensorflow.python.ops import array_ops
        from keras import backend as K

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_sum(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

In [8]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=92)
L2 = resnet_v2(input_tensor=L2, depth=92)
L3 = resnet_v2(input_tensor=L3, depth=92)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
model.compile(loss=[focal_loss(class_nums)], optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [9]:
save_dir = './models'
model_name = 'LCZ_model_3_input.h5'
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
lr_reducr = ReduceLROnPlateau(factor=0.1,cooldown=0, patience=3, min_lr=1e-6)
early = EarlyStopping(monitor='val_acc',patience=5)
calls = [checkpoint, lr_reducr, early]

In [10]:
train_gen = data_generate('./training.h5', batch_size=128, argumentation=True, shuffle=True)
val_gen = data_generate('./validation.h5', batch_size=128, argumentation=True,shuffle=False)

In [11]:
model.fit_generator(train_gen, steps_per_epoch=352366//128+1, epochs=200,  verbose=1,callbacks=calls, validation_data=val_gen,
                   validation_steps=24119//128+1)

Epoch 1/200
2753/2753 [==============================] - 2514s 913ms/step - loss: 4.4128 - acc: 0.7010 - val_loss: 7.8367 - val_acc: 0.5389
Epoch 2/200
2753/2753 [==============================] - 2368s 860ms/step - loss: 2.2667 - acc: 0.7767 - val_loss: 5.2554 - val_acc: 0.5782
Epoch 3/200
2753/2753 [==============================] - 2352s 854ms/step - loss: 1.8800 - acc: 0.8045 - val_loss: 5.8062 - val_acc: 0.5708
Epoch 4/200
2753/2753 [==============================] - 2342s 851ms/step - loss: 1.6898 - acc: 0.8227 - val_loss: 5.3533 - val_acc: 0.5555
Epoch 5/200
2753/2753 [==============================] - 2349s 853ms/step - loss: 1.5758 - acc: 0.8339 - val_loss: 4.6388 - val_acc: 0.5758
Epoch 6/200
2753/2753 [==============================] - 2338s 849ms/step - loss: 1.4901 - acc: 0.8432 - val_loss: 6.3270 - val_acc: 0.6403
Epoch 7/200
2753/2753 [==============================] - 2342s 851ms/step - loss: 1.4208 - acc: 0.8516 - val_loss: 4.6194 - val_acc: 0.6057
Epoch 8/200
2753/275

In [9]:
def data_generate2(data1, data2, data3, batch_size, argumentation = True, shuffle=False):
    while True:
        data_len = data1.shape[0]
        c = [i for i in range(int(data_len/batch_size) + 1)]
        if shuffle:
            np.random.shuffle(c)
        for i in c:
            t = np.random.rand()
            try:
                y_b = data3[i*batch_size:(i+1)*batch_size]
                x_b1 = data1[i * batch_size: (i+1)*batch_size]
                x_b2 = data2[i * batch_size: (i+1)*batch_size]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3], y_b)
            except:
                y_b = data3[i*batch_size:]
                x_b1 = data1[i*batch_size:]
                x_b2 = data2[i*batch_size:]
                x_b3 = np.concatenate([x_b1, x_b2], axis=-1)
                if argumentation:
                    if t < 0.6:
                        k = np.random.choice([2,3,4,5,6])
                        if k == 2:
                            x_b1 = np.rot90(x_b1, k=1, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=1, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=1, axes=(1,2))
                        elif k == 3:
                            x_b1 = np.rot90(x_b1, k=2, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=2, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=2, axes=(1,2))
                        elif k == 4:
                            x_b1 = np.rot90(x_b1, k=3, axes=(1,2))
                            x_b2 = np.rot90(x_b2, k=3, axes=(1,2))
                            x_b3 = np.rot90(x_b3, k=3, axes=(1,2))
                        elif k == 5:
                            x_b1 = x_b1[:,::-1,:,:]
                            x_b2 = x_b2[:,::-1,:,:]
                            x_b3 = x_b3[:,::-1,:,:]
                        elif k == 6:
                            x_b1 = x_b1[:,:,::-1,:]
                            x_b2 = x_b2[:,:,::-1,:]
                            x_b3 = x_b3[:,:,::-1,:]
                yield ([x_b1, x_b2, x_b3], y_b)

In [10]:
valid = h5py.File('./validation.h5', 'r')
valid_s1 = valid['sen1']
valid_s2 = valid['sen2']
valid_label = valid['label']
valid_s1 = np.array(valid_s1)
valid_s2 = np.array(valid_s2)
valid_label = np.array(valid_label)

In [11]:
valid_s1.shape,valid_s2.shape,valid_label.shape

((24119, 32, 32, 8), (24119, 32, 32, 10), (24119, 17))

In [12]:
from sklearn.model_selection import StratifiedKFold, KFold
skf = StratifiedKFold(n_splits=5, random_state=2018)
kf = KFold(n_splits=5, random_state=2018)

In [13]:
y = np.argmax(valid_label, axis=1)

In [12]:
test = h5py.File('./round2_test_a_20190121.h5')
test_s1 = np.array(test['sen1'])
test_s2 = np.array(test['sen2'])

In [15]:
24119*0.8,24119*0.2

(19295.2, 4823.8)

In [16]:
pred = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for i ,(tr_ind, val_ind) in enumerate(skf.split(valid_s1, y)):
    tr1, tr2, trl = valid_s1[tr_ind], valid_s2[tr_ind], valid_label[tr_ind]
    va1, va2, val = valid_s1[val_ind], valid_s2[val_ind], valid_label[val_ind]
    save_dir = './model_2/'
    model_name = 'LCZ_model_FOLD_3_input_%s.h5' % str(i)
    filepath = os.path.join(save_dir, model_name)
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',save_best_only=True, save_weights_only=True)
    lr_reducr = ReduceLROnPlateau(factor=0.1,cooldown=0, patience=3, min_lr=1e-6)
    early = EarlyStopping(monitor='val_acc',patience=5,)
    calls = [checkpoint, lr_reducr, early]
    class_num = [ 256., 1254., 2353.,  849.,  757., 1906.,  474., 3395., 1914.,
        860., 2287.,  382., 1202., 2747.,  202.,  672., 2609.]
    class_num = [int(x*0.8) for x in class_num]
    model.compile(loss=focal_loss(class_num), optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    model.load_weights('./models/LCZ_model_3_input.h5')
    tr_gen = data_generate2(tr1, tr2, trl,argumentation=True,batch_size=128,shuffle=True)
    va_gen = data_generate2(va1, va2, val,argumentation=True,batch_size=128,shuffle=False)
    model.fit_generator(tr_gen, steps_per_epoch=19295//128+1, epochs=200, validation_data=va_gen,validation_steps=4823//128+1, callbacks=calls)
    model.load_weights(filepath)
    for j in range(4):
        te1 = np.rot90(test_s1,k=j,axes=(1,2))
        te2 = np.rot90(test_s2,k=j,axes=(1,2))
        te3 = np.rot90(test_s3,k=j,axes=(1,2))
        pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te3 = test_s3[:,::-1,:,:]
    pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te3 = test_s3[:,:,::-1,:]
    pred += model.predict([te1, te2, te3])

Epoch 1/200
151/151 [==============================] - 236s 2s/step - loss: 2.9846 - acc: 0.6939 - val_loss: 2.2629 - val_acc: 0.7878
Epoch 2/200
151/151 [==============================] - 122s 805ms/step - loss: 2.1662 - acc: 0.7712 - val_loss: 1.9288 - val_acc: 0.8145
Epoch 3/200
151/151 [==============================] - 122s 811ms/step - loss: 1.9271 - acc: 0.8003 - val_loss: 1.7681 - val_acc: 0.8342
Epoch 4/200
151/151 [==============================] - 122s 809ms/step - loss: 1.7898 - acc: 0.8148 - val_loss: 1.6194 - val_acc: 0.8477
Epoch 5/200
151/151 [==============================] - 122s 811ms/step - loss: 1.6981 - acc: 0.8281 - val_loss: 1.5510 - val_acc: 0.8563
Epoch 6/200
151/151 [==============================] - 123s 812ms/step - loss: 1.5957 - acc: 0.8401 - val_loss: 1.4905 - val_acc: 0.8700
Epoch 7/200
151/151 [==============================] - 122s 811ms/step - loss: 1.5535 - acc: 0.8483 - val_loss: 1.4357 - val_acc: 0.8717
Epoch 8/200
151/151 [=======================

Exception ignored in: <generator object data_generate2 at 0x7f9ea7f97db0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f9e900c0048>
RuntimeError: 

Epoch 1/200
151/151 [==============================] - 141s 931ms/step - loss: 2.9953 - acc: 0.6955 - val_loss: 2.1639 - val_acc: 0.7691
Epoch 2/200
151/151 [==============================] - 122s 811ms/step - loss: 2.1845 - acc: 0.7701 - val_loss: 1.8100 - val_acc: 0.8165
Epoch 3/200
151/151 [==============================] - 123s 813ms/step - loss: 1.9648 - acc: 0.7951 - val_loss: 1.6736 - val_acc: 0.8296
Epoch 4/200
151/151 [==============================] - 123s 812ms/step - loss: 1.7997 - acc: 0.8104 - val_loss: 1.6219 - val_acc: 0.8354
Epoch 5/200
151/151 [==============================] - 123s 812ms/step - loss: 1.6966 - acc: 0.8270 - val_loss: 1.4886 - val_acc: 0.8542
Epoch 6/200
151/151 [==============================] - 123s 812ms/step - loss: 1.6323 - acc: 0.8366 - val_loss: 1.4723 - val_acc: 0.8590
Epoch 7/200
151/151 [==============================] - 123s 814ms/step - loss: 1.5540 - acc: 0.8480 - val_loss: 1.3820 - val_acc: 0.8720
Epoch 8/200
151/151 [====================

generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f9c2fa68d58>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f9ea7f97db0>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 144s 955ms/step - loss: 3.0030 - acc: 0.6922 - val_loss: 2.2689 - val_acc: 0.7750
Epoch 2/200
151/151 [==============================] - 123s 817ms/step - loss: 2.1695 - acc: 0.7660 - val_loss: 1.9070 - val_acc: 0.8138
Epoch 3/200
151/151 [==============================] - 123s 815ms/step - loss: 1.9249 - acc: 0.7950 - val_loss: 1.7668 - val_acc: 0.8318
Epoch 4/200
151/151 [==============================] - 123s 816ms/step - loss: 1.7973 - acc: 0.8069 - val_loss: 1.6642 - val_acc: 0.8443
Epoch 5/200
151/151 [==============================] - 123s 815ms/step - loss: 1.6798 - acc: 0.8241 - val_loss: 1.5622 - val_acc: 0.8544
Epoch 6/200
151/151 [==============================] - 123s 815ms/step - loss: 1.5944 - acc: 0.8360 - val_loss: 1.5037 - val_acc: 0.8621
Epoch 7/200
151/151 [==============================] - 123s 815ms/step - loss: 1.5621 - acc: 0.8419 - val_loss: 1.4434 - val_acc: 0.8737
Epoch 8/200
151/151 [====================

Exception ignored in: <generator object data_generate2 at 0x7f9ea7fb3c50>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f9c2fa68d58>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 152s 1s/step - loss: 2.9223 - acc: 0.6942 - val_loss: 2.1929 - val_acc: 0.7810
Epoch 2/200
151/151 [==============================] - 125s 829ms/step - loss: 2.1432 - acc: 0.7703 - val_loss: 1.8798 - val_acc: 0.8094
Epoch 3/200
151/151 [==============================] - 125s 829ms/step - loss: 1.9300 - acc: 0.7956 - val_loss: 1.7284 - val_acc: 0.8270
Epoch 4/200
151/151 [==============================] - 125s 830ms/step - loss: 1.7866 - acc: 0.8108 - val_loss: 1.6226 - val_acc: 0.8426
Epoch 5/200
151/151 [==============================] - 125s 829ms/step - loss: 1.6929 - acc: 0.8229 - val_loss: 1.5252 - val_acc: 0.8525
Epoch 6/200
151/151 [==============================] - 125s 829ms/step - loss: 1.6073 - acc: 0.8380 - val_loss: 1.4491 - val_acc: 0.8693
Epoch 7/200
151/151 [==============================] - 125s 829ms/step - loss: 1.5418 - acc: 0.8478 - val_loss: 1.4158 - val_acc: 0.8718
Epoch 8/200
151/151 [=======================

Exception ignored in: <generator object data_generate2 at 0x7f9ca96bfaf0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_generate2 at 0x7f9ea7fb3c50>
RuntimeError: generator ignored GeneratorExit


Epoch 1/200
151/151 [==============================] - 155s 1s/step - loss: 2.9664 - acc: 0.6935 - val_loss: 2.1966 - val_acc: 0.7824
Epoch 2/200
151/151 [==============================] - 126s 831ms/step - loss: 2.1444 - acc: 0.7702 - val_loss: 1.8767 - val_acc: 0.8196
Epoch 3/200
151/151 [==============================] - 126s 836ms/step - loss: 1.9101 - acc: 0.7981 - val_loss: 1.7260 - val_acc: 0.8418
Epoch 4/200
151/151 [==============================] - 123s 813ms/step - loss: 1.7637 - acc: 0.8166 - val_loss: 1.6014 - val_acc: 0.8372
Epoch 5/200
151/151 [==============================] - 126s 832ms/step - loss: 1.6684 - acc: 0.8265 - val_loss: 1.5320 - val_acc: 0.8540
Epoch 6/200
151/151 [==============================] - 126s 832ms/step - loss: 1.5935 - acc: 0.8388 - val_loss: 1.4680 - val_acc: 0.8571
Epoch 7/200
151/151 [==============================] - 126s 833ms/step - loss: 1.5264 - acc: 0.8469 - val_loss: 1.4496 - val_acc: 0.8698
Epoch 8/200
151/151 [=======================

In [14]:
pred.shape

(4842, 17)

In [15]:
preds = pred / 30

In [16]:
np.sum(preds, axis=1)

array([1.        , 1.        , 0.99999999, ..., 1.        , 1.        ,
       0.99999999])

In [17]:
result = np.zeros((4842, 17))

In [18]:
for i ,ind in enumerate(np.argmax(preds,axis=1)):
    result[i][ind] = 1

In [19]:
result

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [21]:
import pandas as pd
s = pd.DataFrame(result,dtype='int')

In [22]:
s.to_csv('round2_a1.csv', header=None, index=False)